In [ ]:
# stdlib
import os

use_k8s_dev = False
if use_k8s_dev:
    os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
    os.environ["DEV_MODE"] = "True"
    os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
    os.environ["CLUSTER_HTTP_PORT_HIGH"] = "9081"
    os.environ["CLUSTER_HTTP_PORT_LOW"] = "9083"

In [ ]:
# stdlib
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
high_port = os.environ.get("CLUSTER_HTTP_PORT_HIGH", "auto")
low_port = os.environ.get("CLUSTER_HTTP_PORT_LOW", "auto")
print(environment, high_port, low_port)

In [ ]:
# syft absolute
import syft as sy
from syft import get_helpers  # noqa: F401
from syft import test_settings

# Launch server & login

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    reset=True,
    n_consumers=1,
    create_producer=True,
    port=low_port,
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    reset=True,
    n_consumers=1,
    create_producer=True,
    port=high_port,
)

In [ ]:
# start email server here

In [ ]:
low_client = server_low.login(email="info@openmined.org", password="changethis")

In [ ]:
high_client = server_high.login(email="info@openmined.org", password="changethis")

In [ ]:
assert len(high_client.worker_pools.get_all()) == 1
assert len(low_client.worker_pools.get_all()) == 1

# Setup High First

- If you want to use the k8s registry, we submit and build an image, and we scale a worker pool with that image
- If you want to use the k8s registry, do ?????

- helper for launching worker pools

In [ ]:
external_registry = test_settings.get("external_registry", default="docker.io")
external_registry

In [ ]:
result = high_client.api.services.image_registry.add(external_registry)
result

In [ ]:
image_registry_list = high_client.api.services.image_registry.get_all()
image_registry_list

In [ ]:
local_registry = image_registry_list[0]
local_registry

In [ ]:
base_worker_image = high_client.images.get_all()[0]
base_worker_image

In [ ]:
worker_dockerfile = f"""
FROM {str(base_worker_image.image_identifier)}

RUN uv pip install db-dtypes google-cloud-bigquery 

""".strip()
worker_dockerfile

In [ ]:
docker_config = sy.DockerWorkerConfig(dockerfile=worker_dockerfile)
assert docker_config.dockerfile == worker_dockerfile

In [ ]:
submit_result = high_client.api.services.worker_image.submit(
    worker_config=docker_config
)
submit_result

In [ ]:
# get non prebuilt
dockerfile_list = high_client.images.get_all()
worker_image = next(
    (
        image
        for image in dockerfile_list
        if not image.is_prebuilt and image.config.dockerfile == worker_dockerfile
    ),
    None,
)
worker_image

In [ ]:
docker_tag = str(base_worker_image.image_identifier).replace(
    "backend", "worker-bigquery"
)
docker_tag

In [ ]:
if environment == "remote":
    docker_build_result = high_client.api.services.worker_image.build(
        image_uid=worker_image.id,
        tag=docker_tag,
        registry_uid=local_registry.id,
    )
    print(docker_build_result)

In [ ]:
if environment == "remote":
    push_result = high_client.api.services.worker_image.push(worker_image.id)
    print(push_result)

In [ ]:
docker_config = sy.PrebuiltWorkerConfig(tag=docker_tag)
docker_config

In [ ]:
result = high_client.api.services.worker_image.submit(worker_config=docker_config)
worker_image_id = result.value.id
result

In [ ]:
# Update the list
dockerfile_list = high_client.images.get_all()

In [ ]:
# get prebuilt
# dockerfile_list = high_client.images.get_all()
# worker_image = next(
#     (image for image in dockerfile_list if image.is_prebuilt),
#     None,
# )
# worker_image

# TODO: fix
# Similar issue as in non-sync notebooks. Refer to 01-setup-datasite.ipynb

worker_image = next(
    (
        image
        for image in dockerfile_list
        if "worker-bigquery" in str(image.image_identifier)
    ),
    None,
)
worker_image

In [ ]:
assert worker_image

In [ ]:
worker_pool_name = "bigquery-pool"
custom_pool_pod_annotations = {"bigquery-custom-pool": "Pod annotation for bigquery"}
custom_pool_pod_labels = {"bigquery-custom-pool": "Pod_label_for_bigquery"}

In [ ]:
result = high_client.api.services.worker_pool.launch(
    pool_name=worker_pool_name,
    image_uid=worker_image.id,
    num_workers=1,
    pod_annotations=custom_pool_pod_annotations,
    pod_labels=custom_pool_pod_labels,
)
result

In [ ]:
if environment == "remote":
    result = high_client.worker_pools.scale(number=2, pool_name=worker_pool_name)
    print(result)

In [ ]:
assert len(high_client.worker_pools.get_all()) == 2

In [ ]:
widget = sy.sync(from_client=high_client, to_client=low_client, hide_usercode=False)

In [ ]:
widget

In [ ]:
assert len(high_client.worker_pools.get_all()) == 2

In [ ]:
high_client.settings.allow_guest_signup(enable=False)

# Setup Low

In [ ]:
result = low_client.api.services.image_registry.add(external_registry)
result

In [ ]:
docker_config = sy.PrebuiltWorkerConfig(tag=docker_tag)
docker_config

In [ ]:
result = low_client.api.services.worker_image.submit(worker_config=docker_config)
result

In [ ]:
# get prebuilt
dockerfile_list = low_client.images.get_all()
worker_image = next(
    (
        image
        for image in dockerfile_list
        if "worker-bigquery" in str(image.image_identifier)
    ),
    None,
)
worker_image
# worker_image = next(
#     (image for image in dockerfile_list if image.is_prebuilt),
#     None,
# )
# worker_image

In [ ]:
result = low_client.api.services.worker_pool.launch(
    pool_name=worker_pool_name,
    image_uid=worker_image.id,
    num_workers=1,
    pod_annotations=custom_pool_pod_annotations,
    pod_labels=custom_pool_pod_labels,
)
result

In [ ]:
assert len(low_client.worker_pools.get_all()) == 2

In [ ]:
low_client.register(
    email="data_scientist@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="John Doe",
)

In [ ]:
low_client.settings.allow_guest_signup(enable=False)

In [ ]:
assert (
    len(low_client.api.services.user.get_all()) == 2
), "Only DS and Admin should be at low side"
assert (
    len(high_client.api.services.user.get_all()) == 1
), "Only Admin should be at high side"

In [ ]:
# TODO: close email client

In [ ]:
if environment != "remote":
    server_high.land()
    server_low.land()